In [ ]:
# Regression Analysis

import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import OLSInfluence
from statsmodels.stats.diagnostic import het_breuschpagan, acorr_ljungbox
import matplotlib.pyplot as plt
import seaborn as sns

class RegressionDiagnostics:
    def __init__(self, model, X, y, is_time_series=False):
        self.model = model
        self.X = X
        self.y = y
        self.is_time_series = is_time_series
        self.residuals = None

    def _calculate_residuals(self):
        if isinstance(self.model, sm.regression.linear_model.RegressionResultsWrapper):
            self.residuals = self.model.resid
        else:
            raise ValueError("Unsupported model type")

    def homoscedasticity_test(self):
        if self.residuals is None:
            self._calculate_residuals()
        
        _, p_value, _, _ = het_breuschpagan(self.residuals, self.X)
        if p_value > 0.05:
            print("Homoscedasticity Test: Model is homoscedastic")
        else:
            print("Homoscedasticity Test: Model is heteroscedastic")

    def outlier_detection(self):
        if self.residuals is None:
            self._calculate_residuals()

        infl = OLSInfluence(self.model)
        cooks_d = infl.cooks_distance[0]

        # Detect influential points using Cook's distance
        influential_points = np.where(cooks_d > 4 / len(self.X))[0]

        print("Influential Points (indices):", influential_points)

        # Plot leveraged residual plots
        sns.set(style="whitegrid")
        fig, ax = plt.subplots(figsize=(10, 6))
        sns.residplot(infl.hat_matrix_diag, self.residuals, lowess=True, ax=ax, line_kws={"color": "red"})
        ax.set_title("Leveraged Residual Plot")
        plt.show()

    def autocorrelation_test(self):
        if not self.is_time_series:
            print("Autocorrelation Test: Not applicable for non-time series models")
            return
        
        if self.residuals is None:
            self._calculate_residuals()

        lb_test_statistic, lb_p_value = acorr_ljungbox(self.residuals, lags=[10])
        
        if lb_p_value[0] > 0.05:
            print("Autocorrelation Test: Residuals are not autocorrelated")
        else:
            print("Autocorrelation Test: Residuals are autocorrelated")

# Example usage
if __name__ == "__main__":
    # Simulated data for demonstration purposes
    np.random.seed(0)
    n = 100
    X = np.random.rand(n, 2)
    y = 2 * X[:, 0] + 3 * X[:, 1] + np.random.randn(n)
    
    # Regular linear regression model
    model = sm.OLS(y, sm.add_constant(X)).fit()

    # Create an instance of the RegressionDiagnostics class
    diag = RegressionDiagnostics(model, X, y)

    # Perform diagnostics tests
    diag.homoscedasticity_test()
    diag.outlier_detection()

    # Time series example (not real time series data, just for demonstration)
    time_series_resid = np.random.randn(n)
    time_series_model = sm.OLS(time_series_resid, sm.add_constant(X)).fit()

    time_series_diag = RegressionDiagnostics(time_series_model, X, time_series_resid, is_time_series=True)

    time_series_diag.homoscedasticity_test()
    time_series_diag.outlier_detection()
    time_series_diag.autocorrelation_test()
